<a href="https://colab.research.google.com/github/yarnikitin/-/blob/master/%D0%94%D0%97_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Сегментированная воронка конверсии пользователей по полу**

In [44]:
import pandas as pd

In [46]:
user_table = pd.read_csv('user_table.csv')
home_page = pd.read_csv('home_page_table.csv')
search_page = pd.read_csv('search_page_table.csv')
payment_page = pd.read_csv('payment_page_table.csv')
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')

In [47]:
home_page = home_page.rename(columns = {'page' : 'home_page'})
search_page = search_page.rename(columns = {'page' : 'search_page'})
payment_page = payment_page.rename(columns = {'page' : 'payment_page'}) 
payment_confirmation = payment_confirmation.rename(columns = {'page' : 'payment_confirmation'})


In [48]:
df = user_table.merge(home_page, on = 'user_id', how = 'left') \
.merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left') \
.merge(payment_confirmation, on = 'user_id', how = 'left')

df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [49]:
df.shape

(90400, 8)

In [50]:
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)

df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [51]:
dfg =df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().T.reset_index()
dfg.columns.name = None
dfg = dfg.rename(columns = {'index':'Page'})
dfg.head()

,Page,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [55]:
from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Female'],
    name = 'Female',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

fig.add_trace(go.Funnel(
    y = dfg['Page'],
    x = dfg['Male'],
    name = 'Male',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии по гендеру',
    title_x=0.5,
    autosize=False,
    width=900,
    height=450)

fig.show()

# Создать две новые фичи на основе колонки **“date”**

In [56]:
df['date'].dtypes

dtype('O')

In [57]:
df['date'] = pd.to_datetime(df['date'])

In [58]:
df['date'].dtypes

dtype('<M8[ns]')

In [59]:
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month,day_of_week
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2,5
1,756838,2015-01-13,Desktop,Male,1,0,0,0,1,1
2,568983,2015-04-09,Desktop,Male,1,1,0,0,4,3
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2,2
4,537909,2015-01-15,Desktop,Male,1,0,0,0,1,3


# **Топовый месяц по SUM**

In [61]:
df.groupby('month')['payment_confirmation'].sum()

month
1    189
2    173
3     44
4     46
Name: payment_confirmation, dtype: int64

# **Топовый день недели**

In [62]:
df['day_of_week'].loc[df['payment_confirmation'] == 1].value_counts()

0    77
5    70
1    69
3    66
6    62
4    54
2    54
Name: day_of_week, dtype: int64

# **Групприровка покупок по дням недели мужчин и женщин**

In [63]:
df.groupby(['sex', 'day_of_week'])['payment_confirmation'].sum().unstack('sex').sort_values(by = 'Male', ascending = False)

sex,Female,Male
day_of_week,,
3,31,35
0,43,34
5,36,34
1,40,29
4,26,28
6,35,27
2,30,24
